# 00. Setup and Configuration

Parametric QA 실험을 위한 환경 설정 및 구성

## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치 (최초 1회)
# !pip install -r ../requirements.txt

In [ ]:
import sys
from pathlib import Path

# 프로젝트 루트를 path에 추가
PROJECT_ROOT = Path(".").resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
import torch
import yaml
from omegaconf import OmegaConf

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Configuration 로드

In [ ]:
def load_config(config_name: str = "phase1_poc"):
    """Config 파일 로드"""
    config_path = PROJECT_ROOT / "configs" / f"{config_name}.yaml"
    with open(config_path) as f:
        config = OmegaConf.create(yaml.safe_load(f))
    return config

# Phase 1 POC config 로드
config = load_config("phase1_poc")
print(OmegaConf.to_yaml(config))

## 3. 사용 가능한 Config 목록

In [ ]:
configs_dir = PROJECT_ROOT / "configs"
available_configs = list(configs_dir.glob("*.yaml"))

print("Available configurations:")
for cfg in available_configs:
    print(f"  - {cfg.stem}")

## 4. 디렉토리 구조 확인

In [ ]:
import os

# 필요한 디렉토리 생성
dirs_to_create = [
    "data/raw",
    "data/processed",
    "checkpoints",
    "logs",
    "results",
]

for d in dirs_to_create:
    dir_path = PROJECT_ROOT / d
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"Created/Verified: {dir_path}")

## 5. 핵심 모듈 Import 테스트

In [ ]:
# 모듈 import 테스트
try:
    from models.parametric_qa import ParametricQA
    print("✓ ParametricQA imported")
except ImportError as e:
    print(f"✗ ParametricQA import failed: {e}")

try:
    from models.router import CosineSelector, LearnedRouter, AttentionSelector
    print("✓ Router modules imported")
except ImportError as e:
    print(f"✗ Router import failed: {e}")

try:
    from evaluation.metrics import compute_em, compute_f1, compute_recall_at_k
    print("✓ Metrics imported")
except ImportError as e:
    print(f"✗ Metrics import failed: {e}")

## 6. Config Override 예시

In [ ]:
def override_config(config, overrides: dict):
    """Config에 override 적용"""
    config = OmegaConf.to_container(config, resolve=True)
    
    for key, value in overrides.items():
        keys = key.split(".")
        d = config
        for k in keys[:-1]:
            d = d[k]
        d[keys[-1]] = value
    
    return OmegaConf.create(config)

# Override 예시
config_modified = override_config(config, {
    "parametric_qa.z_dim": 512,
    "parametric_qa.m_tokens": 8,
})

print(f"Original z_dim: {config.parametric_qa.z_dim}")
print(f"Modified z_dim: {config_modified.parametric_qa.z_dim}")